In [1]:
import pickle
import numpy as np
import pandas as pd
import joblib

In [2]:
lung1_original = pd.read_csv(
    "/home/suraj/Repositories/foundation-cancer-image-biomarker/data/preprocessing/lung1/annotations.csv"
)
radio_original = pd.read_csv(
    "/home/suraj/Repositories/foundation-cancer-image-biomarker/data/preprocessing/radio/nsclc_radiogenomics.csv"
)

In [16]:
model = joblib.load(
    open(
        "/home/suraj/Repositories/foundation-cancer-image-biomarker/outputs/predictions/task3/HarvardRT/ModelsGenesis/model.pkl",
        "rb",
    )
)

In [17]:
train_df = pd.read_csv(
    "/home/suraj/Repositories/foundation-cancer-image-biomarker/outputs/features/models_genesis/task3/train_features.csv"
)
val_df = pd.read_csv(
    "/home/suraj/Repositories/foundation-cancer-image-biomarker/outputs/features/models_genesis/task3/val_features.csv"
)

lung1_df = pd.read_csv(
    "/home/suraj/Repositories/foundation-cancer-image-biomarker/outputs/features/models_genesis/task3/lung1_features.csv"
)
radio_df = pd.read_csv(
    "/home/suraj/Repositories/foundation-cancer-image-biomarker/outputs/features/models_genesis/task3/radio_features.csv"
)

/tmp/ipykernel_716069/1786420764.py:5: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  radio_df = pd.read_csv("/home/suraj/Repositories/foundation-cancer-image-biomarker/outputs/features/models_genesis/task3/radio_features.csv")


In [18]:
radio_df

,Unnamed: 0,Case ID,Patient affiliation,Age at Histological Diagnosis,Weight (lbs),Gender,Ethnicity,Smoking status,Pack Years,Quit Smoking Year,...,pred_4086,pred_4087,pred_4088,pred_4089,pred_4090,pred_4091,pred_4092,pred_4093,pred_4094,pred_4095
0,0,R01-001,Stanford,79,146,Female,Caucasian,Former,37,1967.0,...,0.018329,0.000000,0.099065,0.000000,0.045692,0.000000,0.083223,0.102294,0.104741,0.228123
1,1,R01-002,Stanford,65,195,Female,Asian,Nonsmoker,NaN,NaN,...,0.008750,0.105171,0.181636,0.188719,0.000000,0.002446,0.022315,0.039966,0.000000,0.029831
2,2,R01-003,VA,65,Not Collected,Male,Caucasian,Current,38,NaN,...,0.046869,0.072895,0.219008,0.059805,0.024826,0.147871,0.053992,0.104248,0.118714,0.353987
3,3,R01-004,VA,67,Not Collected,Male,Caucasian,Former,10,1965.0,...,0.000000,0.010776,0.045867,0.008144,0.049303,0.011580,0.016215,0.000000,0.075688,0.023833
4,4,R01-005,Stanford,84,145,Male,Native Hawaiian/Pacific Islander,Former,20,1951.0,...,0.067861,0.122042,0.088455,0.067606,0.055031,0.175382,0.024027,0.161634,0.075800,0.553660
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,139,R01-141,Stanford,79,155.0,Male,Hispanic/Latino,Former,5.0,1962.0,...,0.000000,0.035994,0.101940,0.001781,0.299988,0.000000,0.246547,0.009068,0.380173,0.028039
140,140,R01-142,Stanford,55,171.0,Female,Caucasian,Former,15.0,1978.0,...,0.110438,0.101647,0.069549,0.026566,0.000000,0.002330,0.066278,0.077242,0.007884,0.007884
141,141,R01-144,Stanford,80,157.0,Male,Asian,Current,10.0,NaN,...,0.000000,0.000000,0.124651,0.087132,0.023512,0.049680,0.087075,0.093381,0.029567,0.101051
142,142,R01-145,Stanford,77,137.0,Male,Asian,Former,40.0,1995.0,...,0.075859,0.073141,0.050384,0.051720,0.039992,0.124564,0.104813,0.176720,0.142489,0.469234


In [19]:
len(train_df), len(val_df), len(lung1_df), len(radio_df)

(203, 88, 421, 144)

In [20]:
lung1_df["surv2yr"] = (lung1_df["Survival.time"] / 365.0 >= 2.0).astype(int)
radio_df["surv2yr"] = (radio_df["Survival.time"] / 365.0 >= 2.0).astype(int)

In [21]:
from sklearn.preprocessing import StandardScaler


data = {"train": {}, "test1": {}, "test2": {}, "val": {}, "target": {}}

feats = train_df.filter(like="pred").columns

sc = StandardScaler()
sc.fit(train_df[feats])
data["train"]["ssl"] = sc.transform(train_df[feats])
data["val"]["ssl"] = sc.transform(val_df[feats])
data["test1"]["ssl"] = sc.transform(lung1_df[feats])
data["test2"]["ssl"] = sc.transform(radio_df[feats])

data["target"]["train"] = train_df["survival"].astype(int)
data["target"]["val"] = val_df["survival"].astype(int)
data["target"]["test1"] = lung1_df["surv2yr"].astype(int)
data["target"]["test2"] = radio_df["surv2yr"].astype(int)

In [22]:
lung1_pred = model.predict_proba(data["test1"]["ssl"])[:, 1]
radio_pred = model.predict_proba(data["test2"]["ssl"])[:, 1]
train_pred = model.predict_proba(data["val"]["ssl"])[:, 1]

In [23]:
from sklearn.metrics import roc_auc_score

roc_auc_score(data["target"]["val"], train_pred)

0.506842105263158

In [24]:
from pathlib import Path

checkpoint_dir = Path("/home/suraj/Repositories/foundation-cancer-image-biomarker/outputs/predictions/task3")

checkpoint_dir.mkdir(parents=True, exist_ok=True)
lung1_df["conf_scores_class_1"] = lung1_pred
radio_df["conf_scores_class_1"] = radio_pred
val_df["conf_scores_class_1"] = train_pred

val_df.to_csv(checkpoint_dir / "HarvardRT" / "modelsgen_features.csv", index=False)
lung1_df.to_csv(checkpoint_dir / "LUNG1" / "modelsgen_features.csv", index=False)
radio_df.to_csv(checkpoint_dir / "RADIO" / "modelsgen_features.csv", index=False)